Collecte de données

Image pokemon de kaggle

licence: Attribution 4.0 International (CC BY 4.0)

Vishal Subbiah (Owner)

data scrapped from: https://pokemondb.net/pokedex/national

Source dataset: https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types?resource=download


In [10]:
from PIL import Image, ExifTags

img = Image.open("data/images/bulbasaur.png")

img.load()  # Needed only for .png EXIF data (see citation above)
print(img.info)

{'gamma': 0.45455, 'srgb': 0, 'dpi': (95.9866, 95.9866), 'transparency': b'\x00\x11\x11\x11\x11\x11"33DDDDDUfffwwwwwwwww\x88\x88\x88\x88\x88\x88\x88\x88\x88\x99\x99\x99\x99\x99\xbb\xbb\xbb\xbb\xbb\xbb\xbb\xbb\xbb\xbb\xbb\xcc\xcc\xcc\xcc\xcc\xcc\xdd\xee\xee\xee\xee\xee\xee\xee\xee\xee'}


In [6]:
import numpy as np 
import pandas as pd
from sklearn import tree
dataset = pd.read_csv('data/pokemon.csv')
result = [
              'Favorite',
              'Favorite',
              'Favorite',
              'Favorite',
              'Favorite',
              'Favorite',
              'NotFavorite',
              'NotFavorite',
              'NotFavorite'
              ]
#Use of decision tree classifiers
resultframe = pd.DataFrame(result, columns=['favorite'])
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataset, resultframe)
print(dataset)

ValueError: could not convert string to float: 'bulbasaur'